# The Traveling Salesman Problem (TSP)
#### The TSP depicts a salesman who has to visit $n$ cities, returning to it's start  (home)  city, whilst not visiting any of the nodes more than once. 
Store data in an adjacency matrix.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math 
from math import floor
from random import randint
import csv as csv
#to shuffle dataframe
from sklearn.utils import shuffle 
from IPython.display import display, HTML
import scipy 
from scipy.misc import comb # comb(n,k, exact=True)
# from scipy import special
# from scipy.special import comb
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [2]:
city_dist = 'city'
def read_data(fileName):
    df = pd.read_csv(fileName)
    return df
    
def check_packaging(df):
    rows, cols = df.shape #size of the data set
    return (rows, cols)

def data_check(df, n=3):
    df_top_n = df.head(n)
    return (df_top_n)

def check_ns(df):
    ns = df.describe()
    return ns


In [3]:
df_cityDist = read_data('%s.csv'%city_dist)
print("rows(%s) x cols(%s) "%check_packaging(df_cityDist))
print()
print("%s"%data_check(df_cityDist))
print()
print(check_ns(df_cityDist))
print()
num_cities = df_cityDist.shape[0]
# df_cityDist.set_index('city')
print('Number of cities including depot: ', num_cities)

rows(5) x cols(5) 

      0     1     2    3   4
0  1000    16    14  100  22
1    16  1000   100   18   4
2    14   100  1000   18   7

                0            1            2            3           4
count     5.00000     5.000000     5.000000     5.000000     5.00000
mean    230.40000   227.600000   227.800000   231.800000   211.20000
std     431.71611   433.470645   433.331513   430.848233   441.03594
min      14.00000     4.000000     7.000000    18.000000     4.00000
25%      16.00000    16.000000    14.000000    18.000000     7.00000
50%      22.00000    18.000000    18.000000    23.000000    22.00000
75%     100.00000   100.000000   100.000000   100.000000    23.00000
max    1000.00000  1000.000000  1000.000000  1000.000000  1000.00000

Number of cities including depot:  5



#### Initilaiization of dataframes which are used to account for verticies visited and calculate distance travelled
NOTE: the $0^{th}$ vertex is the start/depot vertex

In [13]:
distances = pd.DataFrame(np.zeros((num_cities,1)))# The distance from Vertex i to the next vertex
possible = pd.DataFrame(np.zeros((num_cities,1)))# Accounts for the vertices that have been visited
possible.loc[:,:] = True
terminal_set = pd.DataFrame(np.zeros((num_cities,1)))# The set/ordering of vertices that are encountered on the route

distances.rename(columns={0: 'distances'}, inplace=True)
possible.rename(columns={0: 'possible'}, inplace=True)
terminal_set.rename(columns={0: 'terminal_set'}, inplace=True)

display(distances)
display(possible)
display(terminal_set)
display(df_cityDist)


,distances
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


,possible
0,True
1,True
2,True
3,True
4,True


,terminal_set
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


,0,1,2,3,4
0,NaN,16.0,14.0,100.0,22.0
1,16.0,NaN,100.0,18.0,4.0
2,14.0,100.0,NaN,18.0,7.0
3,100.0,18.0,18.0,NaN,23.0
4,22.0,4.0,7.0,23.0,NaN


In [5]:
for i in range(num_cities):
    df_cityDist.iloc[i,i] = np.nan
df_cityDist

,0,1,2,3,4
0,NaN,16.0,14.0,100.0,22.0
1,16.0,NaN,100.0,18.0,4.0
2,14.0,100.0,NaN,18.0,7.0
3,100.0,18.0,18.0,NaN,23.0
4,22.0,4.0,7.0,23.0,NaN


Psuedo-code for nearest neighbour algorithm:
1. From start node (V0), find the nearest neighbour (Vn). In the zeroth row from col 1:end find the smallest distance.
2. Put the next visited neighbour in the terminal set -> T. Add the new vertes  to the the possible set -> P. (Note: this must be assigned to Vn)
3. Do this while D(V0) = 0

In [6]:
distances = pd.DataFrame(np.zeros((num_cities,1)))# The distance from Vertex i to the next vertex
possible = pd.DataFrame(np.zeros((num_cities,1)))# Accounts for the vertices that have been visited
possible.iloc[:,:] = True
terminal_set = pd.DataFrame(np.zeros((num_cities,1)))# The set/ordering of vertices that are encountered on the route

distances.rename(columns={0: 'distances'}, inplace=True)
possible.rename(columns={0: 'possible'}, inplace=True)
terminal_set.rename(columns={0: 'terminal_set'}, inplace=True)
    
def nearest_neighbour(adj_mat, num_cities, distances, possible,terminal_set):#take in the adjacency matrix
    #define possible marix, distance matric and terminal set 
    count = 0
    new_curr = 0 #depot is the first vertex
    while possible.iloc[0,0]== True: 
        if count < num_cities-1:
            curr = new_curr
            nearest_vertex = adj_mat.iloc[curr,1:].idxmin()#gives the minimum values index ie column
            if possible.iloc[nearest_vertex,0]==True: 
                dist = adj_mat.iloc[curr,nearest_vertex]
                distances.iloc[nearest_vertex,0] = dist
                possible.iloc[nearest_vertex,0]=False
                terminal_set.loc[count,0] = nearest_vertex
                adj_mat.iloc[:,nearest_vertex] = np.nan
                new_curr = nearest_vertex
                count = count+1
        elif count == num_cities-1:
            curr = new_curr
            nearest_vertex = 0
            dist = adj_mat.loc[curr,0]
            distaces.iloc[0,0] = dist
            possible.iloc[0,0] = False
            terminal_set.loc[count,0] = 0
    return terminal_set, distances
    

In [7]:
terminal_set, distances = nearest_neighbour(df_cityDist, num_cities, distances, possible, terminal_set)

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [ ]:
adj_mat = df_cityDist
count = 0
new_curr = 0 #depot is the first vertex
# while possible.iloc[0,0]== True: 
if count < num_cities-1:
    curr = new_curr
    nearest_vertex = adj_mat.iloc[curr,1:].idxmin()#gives the minimum values index ie column
    if possible.iloc[nearest_vertex,0]==True: 
        dist = adj_mat.iloc[curr,nearest_vertex]
        distances.iloc[nearest_vertex,0] = dist
        possible.iloc[nearest_vertex,0]=False
        terminal_set.loc[count,0] = nearest_vertex
        adj_mat.iloc[:,nearest_vertex] = np.nan
        new_curr = nearest_vertex
        count = count+1
# elif count == num_cities-1:
# curr = new_curr
# nearest_vertex = 0
# dist = adj_mat.loc[curr,0]
# distaces.iloc[0,0] = dist
# possible.iloc[0,0] = False
# terminal_set.loc[count,0] = 0